# 컨볼루션(Convolution)의 이해

## Convolution : 특정한 패턴의 특징이 어디서 나타나는지를 확인하는 도구
- 이미지 처리에서 특정한 패턴의 특징이 어디서 나타나는지 파악 할 때 사용
- 입력 이미지와 필터를 가지고 Convloution 연산을 하여 2차원 형태의 숫자 집합으로 생성

## feature map : 특징맵
- 특징을 찾기 위해 컨볼루션 필터 하나를 사용하여 특징맵을 생성
- 대상 이미지로부터 필터를 통해 특징을 잡아낸 컨볼루션의 결과를 특징에 대한 위치 정보가 표현된 지도
- 필터 하나로 특징맵 하나를 만들어 냄

In [1]:
# 라이브러리 사용
import tensorflow as tf
import pandas as pd
 
# 데이터를 준비하고
(독립, 종속), _ = tf.keras.datasets.mnist.load_data()
독립 = 독립.reshape(60000, 28, 28, 1) # Convolution 때문에, 현재 이미지는 흑백이라 2차원이므로 3차원으로 변환
종속 = pd.get_dummies(종속)
print(독립.shape, 종속.shape)

(60000, 28, 28, 1) (60000, 10)


- Convolution layer 추가
- Convolution layer에서 필터셋을 몇개 사용할 것인가
- Convolution layer에서 사이즈를 얼마로 할 것인가

tf.keras.layers.Conv2D(필터셋 몇개, 필터셋 크기, activation='swish')  
ex) tf.keras.layers.Conv2D(3, kernel_size=5, activation='swish')

In [2]:
# 모델을 만들고
X = tf.keras.layers.Input(shape=[28, 28, 1]) # 3차원 형태의 관측치를 입력으로 받음, 이유는 만든넘밖에 모름
H = tf.keras.layers.Conv2D(3, kernel_size=5, activation='swish')(X) # 3개의 특징맵 = 3채널의 특징맵
H = tf.keras.layers.Conv2D(6, kernel_size=5, activation='swish')(H) # 6개의 특징맵 = 6채널의 특징맵
H = tf.keras.layers.Flatten()(H) # Flatten을 사용하여 픽셀단위로 한줄로 펼친 후 학습  
H = tf.keras.layers.Dense(84, activation='swish')(H)
Y = tf.keras.layers.Dense(10, activation='softmax')(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')

In [3]:
# 모델을 학습하고
model.fit(독립, 종속, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.3177 - accuracy: 0.9220
Epoch 2/10
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0895 - accuracy: 0.9754
Epoch 3/10
1875/1875 [==============================] - 20s 10ms/step - loss: 0.0608 - accuracy: 0.9827
Epoch 4/10
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0448 - accuracy: 0.9872
Epoch 5/10
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0349 - accuracy: 0.9901
Epoch 6/10
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0278 - accuracy: 0.9922
Epoch 7/10
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0251 - accuracy: 0.9929
Epoch 8/10
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0225 - accuracy: 0.9936
Epoch 9/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0210 - accuracy: 0.9944
Epoch 10/10
1875/1875 [==============================] - 20s 11m

In [8]:
# 모델을 이용합니다. 
pred = model.predict(독립[0:5])
pd.DataFrame(pred).round(2)

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
# 정답 확인
종속[0:5]

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1


In [7]:
# 모델 확인
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 24, 24, 3)         78        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 6)         456       
_________________________________________________________________
flatten (Flatten)            (None, 2400)              0         
_________________________________________________________________
dense (Dense)                (None, 84)                201684    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                850       
Total params: 203,068
Trainable params: 203,068
Non-trainable params: 0
________________________________________________

# 필터의 이해

1. 필터은 3차원 형태로 된 가중치의 모음
 - Convolution 구조에서 학습 하는건 필터
1. 필터 하나는 앞선 레이어의 결과인 "특징맵" 전체를 본다.
 - 전체를 보고 필터셋 하나가 특징맵 하나를 생성
1. 필터 개수 만큼 특징맵을 만든다 
 - tf.keras.layers.Conv2D(3, kernel_size=5, activation='swish')(X) 필터 3개, 사이즈는 5,5  
 - 개별 필터 하나가 3차원 형태, 즉 (5,5,?) 형태가 여러개 존재, 마지막 ?는 특징맵의 채널수와 같음
 - 흑백 이미지면 채널이 1개라서 (5,5,1)
 - 칼라 이미지면 채널이 3개라서 (5,5,3)
 - 그리고 필터 전체의 모양은 4차원 형태가 된다. 따라서 필터 3개의 경우는 (3,5,5,?) 형태가 된다.
 - (3,5,5,?)에서 ?는 앞에서 설명한 특징맵의 채널수와 같음
 
1. 코드분석
```python
H = tf.keras.layers.Conv2D(3, kernel_size=5, activation='swish')(X)
H = tf.keras.layers.Conv2D(6, kernel_size=5, activation='swish')(H)
```

- 흑백 이미지 (28,28,1)
 1. tf.keras.layers.Conv2D(3, kernel_size=5, activation='swish')(X)
   1. 여기서 마지막 1은 흑백이라서 채널이 1개, (칼라면 채널이 3개)
   1. 먼저 (5,5,1) 필터가 3개 준비 되고, 필터 1개당 특징맵 1개를 생성
   1. (5,5,1) 필터 3개에 의하여 특징맵 3개가 생성 -> (24,24,3) 3채널의 특징맵 생성
   1. 특징맵의 사이즈가 28에서 24로 4만큼 줄어듬
   1. 이유는 필터의 사이즈가 5이기 때문에 사이즈에서 1을 뺀 수만큼 사이즈가 감소함

 2. tf.keras.layers.Conv2D(6, kernel_size=5, activation='swish')
   1. 첫번째 Convolution에서 (24,24,3) 3채널의 특징맵 생성
   1. 두번째 Convolution에서(5,5,3) 필터가 6개 준비되고, 필터 1개당 특징맵 1개를 생성
   1. 여기서 마지막이 3인 이유는 첫번째 Convolution에서 나온 특징맵(24,24,3) 채널수와 같은 3으로 생성
   1. (5,5,3) 필터 6개에 의하여 특징맵 6개가 생성 -> (20,20,6) 6채널의 특징맵 생성 
   1. 특징맵 1개를 생성 할 때 필터 1개는 앞의 특징맵 전체 (24,24,3)를 기반으로 새로운 특징맵 1개를 생성